## Collect data

In [1]:
!kaggle competitions download -c aptos2019-blindness-detection

100%|██████████████████████████████████████▉| 8.00G/8.01G [01:35<00:00, 218MB/s]
100%|██████████████████████████████████████| 8.01G/8.01G [01:35<00:00, 89.7MB/s]
 99%|██████████████████████████████████████▍| 1.48G/1.50G [00:10<00:00, 105MB/s]
100%|███████████████████████████████████████| 1.50G/1.50G [00:10<00:00, 149MB/s]
  0%|                                               | 0.00/53.7k [00:00<?, ?B/s]
100%|██████████████████████████████████████| 53.7k/53.7k [00:00<00:00, 46.7MB/s]
  0%|                                               | 0.00/24.5k [00:00<?, ?B/s]
100%|██████████████████████████████████████| 24.5k/24.5k [00:00<00:00, 37.4MB/s]
  0%|                                               | 0.00/28.3k [00:00<?, ?B/s]
100%|██████████████████████████████████████| 28.3k/28.3k [00:00<00:00, 43.3MB/s]


In [3]:
!mv *.csv *.zip data/

In [6]:
!unzip data/train_images.zip -d data/train
!unzip data/test_images.zip -d data/test

Archive:  data/train_images.zip
  inflating: data/train/7b9d519cbd66.png  
  inflating: data/train/ae49cc60f251.png  
  inflating: data/train/b294927b14b0.png  
  inflating: data/train/f4d3777f2710.png  
  inflating: data/train/d74ccc796517.png  
  inflating: data/train/15bed5adde74.png  
  inflating: data/train/f002ce614c59.png  
  inflating: data/train/7335a2d43ada.png  
  inflating: data/train/9b7b6e4db1d5.png  
  inflating: data/train/1968183f0e61.png  
  inflating: data/train/d91273efb92a.png  
  inflating: data/train/5cf9127f251a.png  
  inflating: data/train/370f575adb23.png  
  inflating: data/train/4c17e85686f0.png  
  inflating: data/train/4f7755e74a9e.png  
  inflating: data/train/25e9fd872182.png  
  inflating: data/train/b64e1eef3d63.png  
  inflating: data/train/e7d2c2c3b30f.png  
  inflating: data/train/bfda2fd0533a.png  
  inflating: data/train/224bb938e2dd.png  
  inflating: data/train/d51e5d7484ea.png  
  inflating: data/train/dc6fa1b38b83.png  
  inflating: data/trai

  inflating: data/train/b99c825b93c5.png  
  inflating: data/train/a2811f512c1c.png  
  inflating: data/train/064af6592ba6.png  
  inflating: data/train/8010c931321f.png  
  inflating: data/train/3cdef7c591cc.png  
  inflating: data/train/b8dab47a260e.png  
  inflating: data/train/e5de79795c1d.png  
  inflating: data/train/3c53198519f7.png  
  inflating: data/train/4e54ccfd49b2.png  
  inflating: data/train/26d9576e8043.png  
  inflating: data/train/878e356c8fc9.png  
  inflating: data/train/cb75210abebe.png  
  inflating: data/train/53704c80f0d8.png  
  inflating: data/train/537e50fdf22e.png  
  inflating: data/train/2b3a4a81d748.png  
  inflating: data/train/e1c02f6c3362.png  
  inflating: data/train/0a09aa7356c0.png  
  inflating: data/train/d0b132d2c7ec.png  
  inflating: data/train/c3a82acb7d7a.png  
  inflating: data/train/760b6f4c6d82.png  
  inflating: data/train/bfaa0080ab61.png  
  inflating: data/train/0d744aed4d64.png  
  inflating: data/train/9f37c98b8187.png  
  inflating

  inflating: data/train/1da4a17c18c9.png  
  inflating: data/train/8af50c9d0a86.png  
  inflating: data/train/74418f620068.png  
  inflating: data/train/07122e268a1d.png  
  inflating: data/train/c5a9ebef1517.png  
  inflating: data/train/fce73678f650.png  
  inflating: data/train/2f42e20db938.png  
  inflating: data/train/e4b0df29b96f.png  
  inflating: data/train/55f7f018c61c.png  
  inflating: data/train/7da558d92100.png  
  inflating: data/train/851e40a21f81.png  
  inflating: data/train/b9d0b83d70c3.png  
  inflating: data/train/4b618537d52f.png  
  inflating: data/train/e25ccfe38e44.png  
  inflating: data/train/79ade634c633.png  
  inflating: data/train/f7edc074f06b.png  
  inflating: data/train/8a3eb86ae4bd.png  
  inflating: data/train/a6356a3c5d11.png  
  inflating: data/train/3ac92ac3d65a.png  
  inflating: data/train/1a369baf9ee6.png  
  inflating: data/train/c40976189f22.png  
  inflating: data/train/d41b33fcb94f.png  
  inflating: data/train/3dbc90c7ee7d.png  
  inflating

  inflating: data/train/9ed666e982cd.png  
  inflating: data/train/4ecd1fdd1435.png  
  inflating: data/train/cad5b1a82e60.png  
  inflating: data/train/2ef10194e80d.png  
  inflating: data/train/1ae3c58759fb.png  
  inflating: data/train/705f508d1e42.png  
  inflating: data/train/b9b99dad668d.png  
  inflating: data/train/86722fcd802c.png  
  inflating: data/train/d3dfd0a2dee6.png  
  inflating: data/train/63363410389a.png  
  inflating: data/train/0d9a9896f801.png  
  inflating: data/train/21037f5c7790.png  
  inflating: data/train/573ea80a53be.png  
  inflating: data/train/7d0a871c45db.png  
  inflating: data/train/71c22da3d6c6.png  
  inflating: data/train/51d0034d177d.png  
  inflating: data/train/c704bd669f36.png  
  inflating: data/train/39923b29988a.png  
  inflating: data/train/95a4cc805c7b.png  
  inflating: data/train/b033ab4fb723.png  
  inflating: data/train/bb11db08584a.png  
  inflating: data/train/eb6b1f1c09db.png  
  inflating: data/train/1120f6d08d95.png  
  inflating

  inflating: data/train/0e82bcacc475.png  
  inflating: data/train/eabf421f94d0.png  
  inflating: data/train/7b8c78b41c0d.png  
  inflating: data/train/910bfd38e2f5.png  
  inflating: data/train/7d48f8cdfb69.png  
  inflating: data/train/b7f0bc7d399e.png  
  inflating: data/train/25d069089c5e.png  
  inflating: data/train/9fefe2b44795.png  
  inflating: data/train/3dfc50108072.png  
  inflating: data/train/e3e490babc0c.png  
  inflating: data/train/b8297a2291f5.png  
  inflating: data/train/a19507501b40.png  
  inflating: data/train/df0886f1e76b.png  
  inflating: data/train/215d2b7c3fde.png  
  inflating: data/train/e811f39a1243.png  
  inflating: data/train/ce754234d760.png  
  inflating: data/train/958c1fa044ba.png  
  inflating: data/train/5e5275ddee29.png  
  inflating: data/train/02dda30d3acf.png  
  inflating: data/train/6a91eb157f47.png  
  inflating: data/train/6b3860e8f64f.png  
  inflating: data/train/d8cdb7d7283a.png  
  inflating: data/train/4aa07d720638.png  
  inflating

...

  inflating: data/test/f08c01a8a480.png  
  inflating: data/test/33712f0c55f1.png  
  inflating: data/test/6825ccf0affe.png  
  inflating: data/test/73a0917d732b.png  
  inflating: data/test/c57e87ac7d8b.png  
  inflating: data/test/3f42e760cc3e.png  
  inflating: data/test/39a7cac6ac9c.png  
  inflating: data/test/8c49d9de02d6.png  
  inflating: data/test/ec71f67e2cd1.png  
  inflating: data/test/4d6333d41069.png  
  inflating: data/test/636997a49fcb.png  
  inflating: data/test/be644233f515.png  
  inflating: data/test/214e78f46682.png  
  inflating: data/test/b5ef4a1296d4.png  
  inflating: data/test/481657080d8b.png  
  inflating: data/test/4f985a7320d9.png  
  inflating: data/test/c39c40be80e9.png  
  inflating: data/test/3382ab75cc96.png  
  inflating: data/test/570982b0dbf2.png  
  inflating: data/test/810b1ef05d20.png  
  inflating: data/test/db169abd047e.png  
  inflating: data/test/bab03e16c029.png  
  inflating: data/test/a48d59891a94.png  
  inflating: data/test/9bd683e1632

  inflating: data/test/f7dd2700a3d5.png  
  inflating: data/test/9eac62719123.png  
  inflating: data/test/87487c4c4ddc.png  
  inflating: data/test/828ad210e3e2.png  
  inflating: data/test/3a2f02646df3.png  
  inflating: data/test/aaa2cf8d02ef.png  
  inflating: data/test/9003ab48ebb8.png  
  inflating: data/test/1278c303e03e.png  
  inflating: data/test/ee76112096cf.png  
  inflating: data/test/3832a69f943a.png  
  inflating: data/test/133f96f06473.png  
  inflating: data/test/6ea2fc471536.png  
  inflating: data/test/aa7e34315323.png  
  inflating: data/test/d073f7f99d46.png  
  inflating: data/test/d995728f0e26.png  
  inflating: data/test/13065b0ff1e0.png  
  inflating: data/test/dfff4dd8e2a3.png  
  inflating: data/test/51910bcb980d.png  
  inflating: data/test/5b7e38adccf3.png  
  inflating: data/test/58ab162bfc89.png  
  inflating: data/test/c80eebab468d.png  
  inflating: data/test/d8468d108c02.png  
  inflating: data/test/9d9de8c9afb5.png  
  inflating: data/test/2fb539602f5

  inflating: data/test/42a390ee5583.png  
  inflating: data/test/a2d894364873.png  
  inflating: data/test/1a18d78ffb56.png  
  inflating: data/test/68d97772bd3a.png  
  inflating: data/test/fef8e645d030.png  
  inflating: data/test/88fda3e06e05.png  
  inflating: data/test/74b0edef5063.png  
  inflating: data/test/f7cb2e6212b2.png  
  inflating: data/test/8938eb4d9c63.png  
  inflating: data/test/da763f0c1a0f.png  
  inflating: data/test/644f1ba18d8b.png  
  inflating: data/test/b21c1ff1bb64.png  
  inflating: data/test/58ec1c8ed8b1.png  
  inflating: data/test/bc7419fa4437.png  
  inflating: data/test/b430b6148d0b.png  
  inflating: data/test/3b9ee705c750.png  
  inflating: data/test/774cafde22b5.png  
  inflating: data/test/51a3e9b6a1c0.png  
  inflating: data/test/b29bd35acaf6.png  
  inflating: data/test/62d9da36bd9e.png  
  inflating: data/test/c19b684d1f54.png  
  inflating: data/test/99c6cf808e47.png  
  inflating: data/test/106dac320289.png  
  inflating: data/test/8a177340ac5

  inflating: data/test/8dd1c698ae62.png  
  inflating: data/test/d490d6bf7a9c.png  
  inflating: data/test/f32613d8dd7f.png  
  inflating: data/test/5a311e3e8979.png  
  inflating: data/test/0f96e3e4a5a0.png  
  inflating: data/test/273a341b5144.png  
  inflating: data/test/eea7f3cdc9dd.png  
  inflating: data/test/156350bc41c9.png  
  inflating: data/test/0299d97f31f7.png  
  inflating: data/test/8cf4258bd346.png  
  inflating: data/test/ef514c87f4ec.png  
  inflating: data/test/1a0a62061725.png  
  inflating: data/test/5ab71d422459.png  
  inflating: data/test/afb7907b77d9.png  
  inflating: data/test/14ab77198610.png  
  inflating: data/test/e31794f248df.png  
  inflating: data/test/85105e9ab635.png  
  inflating: data/test/8c0f725874af.png  
  inflating: data/test/4810eb8e7c88.png  
  inflating: data/test/93c1af8b625c.png  
  inflating: data/test/1ff6e5ec40b4.png  
  inflating: data/test/0268f4382c67.png  
  inflating: data/test/ef8c405f0c7b.png  
  inflating: data/test/ae23bd4b0b5

  inflating: data/test/4d80c835d862.png  
  inflating: data/test/4c49594c40c9.png  
  inflating: data/test/7c0c20cd8114.png  
  inflating: data/test/10c9d368f264.png  
  inflating: data/test/1b4843340486.png  
  inflating: data/test/464c22b720ea.png  
  inflating: data/test/316f294f35ad.png  
  inflating: data/test/716b001ba7a4.png  
  inflating: data/test/c8f81210b738.png  
  inflating: data/test/b7b9839a025b.png  
  inflating: data/test/f7f8dd6cd6bd.png  
  inflating: data/test/178bf0bd8902.png  
  inflating: data/test/616a989990b0.png  
  inflating: data/test/3fa9277a3780.png  
  inflating: data/test/c1a060f49e24.png  
  inflating: data/test/cba1fb3d80e0.png  
  inflating: data/test/dd525bbb2ca3.png  
  inflating: data/test/2109169f580e.png  
  inflating: data/test/6c96703bc565.png  
  inflating: data/test/e4c510a40d9c.png  
  inflating: data/test/fc5ea7f74c48.png  
  inflating: data/test/f7de9615aea3.png  
  inflating: data/test/9c27b212d50b.png  
  inflating: data/test/7eeed2e26c9

In [7]:
!rm data/*.zip

rm: cannot remove 'data/*.zip': No such file or directory
